### Strong and Weak References

### Сильные и слабые ссылки

Для начала давайте вернём функцию, которую мы можем использовать для определения количества ссылок объекта по идентификатору:

In [1]:
import ctypes

def ref_count(address):
    return ctypes.c_long.from_address(address).value

Обратите внимание, что при этом учитываются **сильные** ссылки на этот объект.

До сих пор мы всегда работали с сильными референсами.

In [2]:
class Person:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return f'Person(name={self.name})'

In [3]:
p1 = Person('Guido')
p2 = p1

В этом случае и `p1`, и `p2` являются **сильными** ссылками на один и тот же экземпляр `Person` (*Guido*).

In [4]:
p1_id = id(p1)
p2_id = id(p2)

In [5]:
p1_id == p2_id, ref_count(p1_id)

(True, 2)

Итак, у нас есть две сильные ссылки. Если мы удалим одну из них:

In [6]:
del p2

Теперь у нас должно быть сильное количество ссылок, равное `1`:

In [7]:
ref_count(p1_id)

1

Последнюю ссылку можно удалить:

In [8]:
del p1

Теперь наша функция подсчета ссылок больше не будет работать, поскольку последняя ссылка на объект по этому адресу памяти была удалена, и этот адрес памяти теперь не имеет смысла:

In [9]:
ref_count(p1_id)

-370994432650002694

Таким образом, сборщик мусора уничтожит любой объект, **сильный** счетчик ссылок которого упадет до `0`.

Существует еще один тип ссылки на объект, который мы можем использовать и который **не** влияет на (сильное) количество ссылок. Такие ссылки называются **слабыми ссылками**.

Слабые ссылки на объекты в Python можно создавать с помощью модуля `weakref`:

In [10]:
import weakref

In [11]:
p1 = Person('Guido')

In [12]:
p1_id = id(p1)

In [13]:
ref_count(p1_id)

1

Теперь давайте сделаем еще одну весомую ссылку:

In [14]:
p2 = p1

In [15]:
ref_count(p1_id)

2

И наконец, сделаем слабую ссылку на тот же объект:

In [16]:
weak1 = weakref.ref(p1)

Давайте еще раз посмотрим на количество ссылок:

In [17]:
ref_count(p1_id)

2

Как видите, это по-прежнему «2».

Объект `weak1` является слабым ссылочным объектом:

In [18]:
weak1

<weakref at 0x7fbae83667c8; to 'Person' at 0x7fbae8359908>

Как вы можете видеть из представления, это его собственный объект, но он указывает на тот же объект, на который в данный момент указывает `p1`:

In [19]:
hex(p1_id)

'0x7fbae8359908'

Таким образом, `weak1` не является экземпляром `Person`:

In [20]:
weak1 is p1

False

In [21]:
ref_count(p1_id)

2

Но его можно вызвать (поэтому он реализует метод `__call__`), который вернет объект, на который он указывает:

In [22]:
weak1() is p1

True

И мы видим объект, на который он указывает:

In [23]:
print(weak1())

Person(name=Guido)


Теперь нам нужно быть осторожными, если бы мы не использовали оператор `print`, Jupyter удерживал бы сильные ссылки на наш объект! Обязательно используйте `print` при использовании Jupyter...

Таким образом, наше количество ссылок по-прежнему должно быть равно `2`:

In [24]:
ref_count(p1_id)

2

Еще одно предостережение, если мы сделаем это:

In [25]:
p3 = weak1()

`p3` теперь является сильной ссылкой на любой объект, возвращаемый `weak1()`! В этом случае наш *Guido* `Person`:

In [26]:
p1 is p3

True

In [27]:
ref_count(p1_id)

3

И как вы видите, теперь у нас есть три весомых ссылки.

Сколько у нас слабых ссылок? У нас должно быть только `1`.

Мы можем увидеть, сколько слабых ссылок существует из некоторого объекта, используя функцию `getweakrefcount` в модуле `weakref`:

In [28]:
weakref.getweakrefcount(p1), ref_count(p1_id)

(1, 3)

Другой способ получения строгого количества ссылок — в модуле `sys`:

In [29]:
import sys

In [30]:
sys.getrefcount(p1)

4

Но вы заметите одну вещь: количество ссылок увеличивается на `1` — это потому, что нам нужно передать сам объект в качестве дополнительного аргумента, так что это очень сильная ссылка! (поэтому, по сути, всегда вычитайте `1` из количества ссылок, чтобы получить истинное количество ссылок)

Теперь давайте удалим некоторые сильные ссылки:

In [31]:
del p3
del p2

In [32]:
ref_count(p1_id)

1

Количество наших сильных ссылок сократилось до 1, и у нас все еще есть одна слабая ссылка (`weak1`).

Теперь удалим последнюю сильную ссылку:

In [33]:
del p1

Наше сильное количество ссылок упало до «0», поэтому сборщик мусора уничтожил объект.

Так что же случилось с нашей слабой ссылкой?

In [34]:
weak1

<weakref at 0x7fbae83667c8; dead>

Слабый объект ссылки все еще существует, но объект, на который он указывает, **dead**.

На самом деле, если мы попытаемся получить объект, то получим `None`:

In [35]:
obj = weak1()

In [36]:
obj is None

True

Как видите, наличие слабой ссылки не помешало нашему объекту быть уничтоженным после того, как все сильные ссылки исчезли.

Обратите внимание, что не каждый объект в Python поддерживает слабые ссылки. Многие встроенные типы не поддерживают:

In [37]:
l = [1, 2, 3]
try:
    w = weakref.ref(l)
except TypeError as ex:
    print(ex)

cannot create weak reference to 'list' object


In [38]:
l = {'a': 1}
try:
    w = weakref.ref(l)
except TypeError as ex:
    print(ex)

cannot create weak reference to 'dict' object


In [39]:
l = 100
try:
    w = weakref.ref(l)
except TypeError as ex:
    print(ex)


cannot create weak reference to 'int' object


In [40]:
l = 'python'
try:
    w = weakref.ref(l)
except TypeError as ex:
    print(ex)

cannot create weak reference to 'str' object


Но наши пользовательские классы это делают, и это то, что нам здесь нужно.

Для наших дескрипторов данных мы хотим использовать объекты экземпляров как ключи в нашем словаре. Но как мы видели ранее, хранение самого объекта как ключа может привести к утечкам памяти. Поэтому вместо этого мы будем хранить слабые ссылки на объект в словаре.

Мы могли бы использовать наш собственный словарь, но `weakref` также предоставляет специализированный тип словаря, который будет хранить слабую ссылку на объект, используемый в качестве ключа:

In [41]:
p1 = Person('Guido')

In [42]:
d = weakref.WeakKeyDictionary()

In [43]:
ref_count(id(p1))

1

In [44]:
weakref.getweakrefcount(p1)

0

In [45]:
d[p1] = 'Guido'

Теперь обратите внимание на количество ссылок:

In [46]:
ref_count(id(p1)), weakref.getweakrefcount(p1)

(1, 1)

У нас по-прежнему только одна сильная ссылка, но теперь у нас есть и слабая ссылка на `p1`! Эта слабая ссылка находится в `WeakKeyDictionary`.

Мы легко можем увидеть слабые ссылки, содержащиеся в этом словаре:

In [47]:
hex(id(p1)), list(d.keyrefs())

('0x7fbae83635c0',
 [<weakref at 0x7fbae8381958; to 'Person' at 0x7fbae83635c0>])

Теперь посмотрите, что происходит со словарем, когда мы удаляем последнюю сильную ссылку на `p1`:

In [48]:
del p1

In [49]:
list(d.keyrefs())

[]

Он был автоматически удален, когда объект, на который он указывал (слабо), был уничтожен сборщиком мусора!

Теперь будьте осторожны, вы можете использовать только те ключи в `WeakKeyDictionary`, на которые Python может создавать слабые ссылки:

Так что это не сработает:

In [50]:
try:
    d['python'] = 'test'
except TypeError as ex:
    print(ex)

cannot create weak reference to 'str' object


Кроме того, даже если мы используем слабую ссылку в качестве ключа в словаре, объект все равно должен быть **хешируемым**.

Давайте рассмотрим пример этого:

In [51]:
class Person:
    def __init__(self, name):
        self.name = name

    def __eq__(self, other):
        return isinstance(other, Person) and self.name == other.name

Теперь `Person` больше не подлежит хешированию:

In [52]:
p1 = Person('Guido')
p2 = Person('Guido')

In [53]:
p1 == p2

True

In [54]:
try:
    hash(p1)
except TypeError as ex:
    print(ex)

unhashable type: 'Person'


И поэтому мы не можем использовать его в качестве ключа в нашем `WeakKeyDictionary`:

In [55]:
try:
    d[p1] = 'Guido'
except TypeError as ex:
    print(ex)

unhashable type: 'Person'


Итак, мы, конечно, можем использовать объекты `WeakKeyDictionary` в наших дескрипторах данных, но это будет работать только с хешируемыми объектами. В следующих лекциях мы рассмотрим, как использовать `WeakKeyDictionary` в качестве механизма хранения для наших дескрипторов данных, а также как справиться с проблемой нехешируемости.